In [82]:
# Libraries
import pandas as pd
import requests as rq
import re as re

In [85]:
# Get all links of the indicators
df_links = pd.read_excel("Data-sheet-STUDENTS.xlsx", sheet_name="Data sources", header=None, usecols="C", skiprows=3, na_values="")
df_links = df_links.dropna()
links = []
for index, link in df_links[2].items():
    # We don't handle other source for now
    if "data.worldbank" not in link:
        continue
        
    # Match the useful indicator in the link
    match = re.search("indicator/([\.|\w]*)", link)
    links.append(match.group(1))

In [86]:
# Get countries data
df = pd.read_excel('Data-sheet-STUDENTS.xlsx', header=2, index_col=0)

In [87]:
# Store the info of Country - ISO Code
iso_codes = {}

In [88]:
# Get countries name for API request - Get ISO code
for index, name in df['Country (according to CN Gov):'].items():
    
    # Clean the data
    cleaned_name = name.split('(')[0].strip()
    if name == "Thailiand" :
        cleaned_name = "Thailand"
    if name == "The United Arab Emirates":
        cleaned_name = "United Arab Emirates"
    
    # We don't want to redo the request if the name already exists
    if name in iso_codes :
        continue
         
    # Request part
    apiURL = "https://restcountries.eu/rest/v2/name/" + cleaned_name
    resp = rq.get(apiURL)
        
    # Error handler
    if(resp.status_code != 200) :
        print("STATUS_CODE ERROR", resp.status_code, resp.text)
        continue
        
    # Response handler
    data = resp.json()
    alphaCode = data[0]['alpha3Code']
    if alphaCode == "IOT":
        alphaCode = "IND"
    iso_codes[name] = alphaCode

In [89]:
# Create new column from response in the data frame
df['isocode'] = df['Country (according to CN Gov):'].apply(lambda x: iso_codes[x])

In [90]:
# Store the info of ISO Code + the other indicators
countries_inds = {}

In [98]:
# Get all columns name of the indicators
for indic in links:
    for country in iso_codes:
        
        # We don't handle other source for now
        if "data.worldbank" not in link:
            continue

        # Request part
        apiURL = "http://api.worldbank.org/v2/country/"+ iso_codes[country] +"/indicator/"+ indic +"?format=json&mrv=1"
        resp = rq.get(apiURL)

        # Error handler
        if(resp.status_code != 200) :
            print("STATUS_CODE ERROR", resp.status_code, resp.text)
            print("indicator : " + indic + " - country : " + iso_codes[country])
            continue

        # Response handler
        print("indicator : " + indic + " - country : " + country)
        data = resp.json()
        print(iso_codes[country])
        print(data)
        if data[1] is None:
            tax_ind = 0
        else :
            tax_ind = data[1][0]['value']
        countries_inds[iso] = tax_ind
        break
    break
countries_inds

indicator : IC.REG.COST.PC.ZS - country : Afghanistan
AFG
[{'page': 1, 'pages': 1, 'per_page': 50, 'total': 1, 'sourceid': '2', 'lastupdated': '2019-09-27'}, [{'indicator': {'id': 'IC.REG.COST.PC.ZS', 'value': 'Cost of business start-up procedures (% of GNI per capita)'}, 'country': {'id': 'AF', 'value': 'Afghanistan'}, 'countryiso3code': 'AFG', 'date': '2018', 'value': 6.4, 'unit': '', 'obs_status': '', 'decimal': 1}]]


NameError: name 'iso' is not defined

In [24]:
# Get countries TAX indicator 
for index, iso in df['isocode'].items():
    # We don't want to redo the request if the name already exists
    if iso in countries_inds:
        continue
    
    # Request part
    apiURL = "http://api.worldbank.org/v2/country/"+ iso +"/indicator/GC.TAX.IMPT.ZS?format=json&mrv=1"
    resp = rq.get(apiURL)
        
    # Error handler
    if(resp.status_code != 200) :
        print("STATUS_CODE ERROR", resp.status_code, resp.text)
        print(iso)
        continue
        
    # Response handler
    data = resp.json()
    if data[1] is None:
        tax_ind = 0
    else :
        tax_ind = data[1][0]['value']
    countries_inds[iso] = tax_ind

NameError: name 'df' is not defined

In [75]:
# Add the TAX indicator values to the current data frame
for index, tax in df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].items():
    if pd.isnull(df.loc[index, 'IMF WB\nCustoms and other import duties (% of tax revenue) 2016']):
        df.loc[index, 'IMF WB\nCustoms and other import duties (% of tax revenue) 2016'] = tax_inds.loc[index, 'tax']

AttributeError: 'dict' object has no attribute 'loc'

In [67]:
df
#df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'] = df['isocode'].apply(lambda x: tax_inds[x])

,Country (according to CN Gov):,Date of research inquiry,HF Index of Economic Freedom (Scale: 1-100),Heritage Foundation Index of Economic Freedom (Global Ranking),Bertelsmann Transformation index (BTI),Bertelsmann Transformation index (BTI) - Ranking,IMF WB Customs and other import duties (% of tax revenue) 2016,WB IMF Non-performing loans to total gross loans ratio (%),Moody's Socvereign Ratings List,S&P Sovereign Ratings List,...,WB Ease of Doing Business Index (1 = most business-friendly regulations),WB Logistics Performance Index (LPI) (range: 1-5),IFDI Net Inflows (% of GDP),ED National Electric Power Consumption TwH,"WHO Current Health Expenditure p.c., PPP USD",WB Research Expenditure % of GDP,WIPO Trademark applications,WIPO Patent Applications,isocode,tax
1,Afghanistan,NaT,NaN,NaN,NaN,NaN,AFG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AFG,35.203141
2,Albania,NaT,NaN,NaN,NaN,NaN,ALB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALB,2.216459
3,Armenia,NaT,NaN,NaN,NaN,NaN,ARM,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARM,6.275395
4,Azerbaijan,NaT,NaN,NaN,NaN,NaN,AZE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AZE,7.571847
5,Bahrain,NaT,NaN,NaN,NaN,NaN,BHR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BHR,27.386091
6,Bangladesh,NaT,NaN,NaN,NaN,NaN,BGD,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BGD,28.923008
7,Belarus,NaT,NaN,NaN,NaN,NaN,BLR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLR,8.200619
8,Bhutan,NaT,NaN,NaN,NaN,NaN,BTN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BTN,2.840501
9,Bosnia and Herzegovina,NaT,NaN,NaN,NaN,NaN,BIH,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BIH,0.000155
10,Brunei,NaT,NaN,NaN,NaN,NaN,BRN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRN,0.000000
